In [1]:
# AutoGluon for Binding Free Energy Prediction
import os
import json
import ast
from itertools import product

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from autogluon.tabular import TabularPredictor

descriptor_excel = (
    '/path/to/host_guest_features_SI.xlsx'
)

df = pd.read_excel(descriptor_excel, sheet_name='Features').reset_index(drop=True)

ID = df['ID']
y = df['deltaG']
y_array = y.values
ID_array = ID.values
type_labels = df['Type'].astype(str).values

shared_features = [
    'GuestLogP', 'GuestTPSA', 'PoreDiameter', 'PoreVolume',
    'GuestCavityRatio', 'HostMPI', 'GuestMPI',
    'HostPositiveSA', 'HostNegativeSA', 'GuestPositiveSA', 'GuestNegativeSA',
    'HostPolarSA', 'HostNonpolarSA', 'GuestPolarSA', 'GuestNonpolarSA',
    'InteractionFP', 'ESPFitness',
]


def build_feature_matrix(df, fp_blocks, shared_features):
    feature_cols = fp_blocks + shared_features
    X_parts = []

    for col in feature_cols:
        if col not in df.columns:
            raise KeyError(f"Column '{col}' not found in DataFrame")

        s = df[col]
        first_val = s.iloc[0]

        if isinstance(first_val, (list, tuple, np.ndarray)):
            m = pd.DataFrame(s.tolist(), index=df.index)
            m = m.add_prefix(col + "_")
            X_parts.append(m)
            continue

        if isinstance(first_val, str):
            parsed_ok = False
            try:
                tmp = ast.literal_eval(first_val)
                if isinstance(tmp, (list, tuple, np.ndarray)):
                    seq_series = s.apply(
                        lambda x: ast.literal_eval(x) if isinstance(x, str) else x
                    )
                    m = pd.DataFrame(seq_series.tolist(), index=df.index)
                    m = m.add_prefix(col + "_")
                    X_parts.append(m)
                    parsed_ok = True
            except (ValueError, SyntaxError):
                parsed_ok = False

            if parsed_ok:
                continue
        X_parts.append(s.to_frame())

    X_feat = pd.concat(X_parts, axis=1)
    return X_feat


base_model_root = (
    './AutoGluonModels'
)

os.makedirs(base_model_root, exist_ok=True)

all_summary_records = []

n_splits_outer = 5

fp_blocks = ['HostMACCSKey', 'GuestMACCSKey']
X_feat = build_feature_matrix(df, fp_blocks, shared_features)
X_feat = X_feat.reset_index(drop=True)
assert X_feat.shape[0] == len(df), "Row count mismatch."

print(X_feat.shape)
train_data = X_feat.copy()
train_data['deltaG'] = y
print(X_feat.columns)

(876, 363)
Index(['HostMACCSKey_0', 'HostMACCSKey_1', 'HostMACCSKey_2', 'HostMACCSKey_3',
       'HostMACCSKey_4', 'HostMACCSKey_5', 'HostMACCSKey_6', 'HostMACCSKey_7',
       'HostMACCSKey_8', 'HostMACCSKey_9',
       ...
       'InteractionFP_4', 'InteractionFP_5', 'InteractionFP_6',
       'InteractionFP_7', 'InteractionFP_8', 'InteractionFP_9',
       'InteractionFP_10', 'InteractionFP_11', 'InteractionFP_12',
       'ESPFitness'],
      dtype='object', length=363)


In [2]:
# Feature Selection
train_df = X_feat.copy()
train_df['deltaG'] = y

predictor_full = TabularPredictor(
    label='deltaG',
    problem_type='regression',
    eval_metric='r2'
).fit(
    train_data=train_df,
    presets='good_quality',
    num_cpus=24,
    num_gpus=0,
    num_stack_levels=0,
    auto_stack=False,
)

fi = predictor_full.feature_importance(train_df)
fi_sorted = fi.sort_values('importance', ascending=False)
fi_sorted = fi_sorted[fi_sorted['importance'] > 0]

cum_importance = fi_sorted['importance'].cumsum() / fi_sorted['importance'].sum()
selected_features = cum_importance[cum_importance <= 0.95].index.tolist()

No path specified. Models will be saved in: "AutogluonModels/ag-20260110_044856"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.10.18
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #63~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Apr 22 19:00:15 UTC 2
CPU Count:          32
Memory Avail:       80.17 GB / 94.10 GB (85.2%)
Disk Space Avail:   837.70 GB / 3753.99 GB (22.3%)
Presets specified: ['good_quality']
Using hyperparameters preset: hyperparameters='light'
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "/mnt/4TBStorage1/Desktop/Papers/HostGuestModel/HostGuestModel_DataFile/BindingPrediction/ver_SupraBank/MLDeltaG/MLDeltaGCodes/AutogluonModels/ag-20260110_044856"
Train Data Rows:    876
Train Data Columns: 363
Label Column:       deltaG
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitti

[1000]	valid_set's l2: 1.50075	valid_set's r2: 0.805408


	0.806	 = Validation score   (r2)
	3.31s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM ... Training model for up to 3595.30s of the 3595.30s of remaining time.
	Fitting with cpus=24, gpus=0, mem=0.0/80.1 GB


[1000]	valid_set's l2: 1.76815	valid_set's r2: 0.770735


	0.7708	 = Validation score   (r2)
	5.36s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForestMSE ... Training model for up to 3589.92s of the 3589.92s of remaining time.
	Fitting with cpus=24, gpus=0
	0.7172	 = Validation score   (r2)
	0.35s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: CatBoost ... Training model for up to 3589.51s of the 3589.51s of remaining time.
	Fitting with cpus=24, gpus=0
	0.7854	 = Validation score   (r2)
	28.59s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: ExtraTreesMSE ... Training model for up to 3560.89s of the 3560.89s of remaining time.
	Fitting with cpus=24, gpus=0
	0.7751	 = Validation score   (r2)
	0.3s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: NeuralNetFastAI ... Training model for up to 3560.54s of the 3560.54s of remaining time.
	Fitting with cpus=24, gpus=0, mem=0.0/79.7 GB
	0.74	 = Validation score   (r2)
	2.11s	 = Training   runtime
	0.01s	 = Validation ru

In [3]:
print(f"Selected {len(selected_features)} features out of {X_feat.shape[1]}")

Selected 101 features out of 363


In [ ]:
from sklearn.model_selection import StratifiedKFold

all_summary_records = []

n_splits_outer = 5

skf = StratifiedKFold(
    n_splits=n_splits_outer,
    shuffle=True,
    random_state=1000
)

selected_X_feat = X_feat.copy()
for feat in selected_X_feat.columns:
    if feat not in selected_features:
        selected_X_feat = selected_X_feat.drop([feat], axis=1)

print(f'Shape of selected_feat: {selected_X_feat.shape}')

model_root_path = os.path.join(base_model_root, '101Features_good_5bag_5fold')
os.makedirs(model_root_path, exist_ok=True)

with open(os.path.join(model_root_path, 'selected_features.txt'), 'w') as f:
    f.writelines('\n'.join(selected_features))

kf_results = {}
excel_out = os.path.join(model_root_path, 'PredictionResults_5fold.xlsx')
if os.path.exists(excel_out):
    print(f'{model_root_path} has already been trained.')
    quit()

type_labels = df['Type'].astype(str).values

ID = df['ID']
y = df['deltaG']
y_array = y.values
ID_array = ID.values

# ========== 5-fold==========
for fold, (train_idx, test_idx) in enumerate(skf.split(selected_X_feat, type_labels), start=1):
    print(f"\n===== Training fold {fold} / {n_splits_outer} =====")

    X_train = selected_X_feat.iloc[train_idx].copy()
    X_test = selected_X_feat.iloc[test_idx].copy()
    y_train = y_array[train_idx]
    y_test = y_array[test_idx]
    id_train = ID_array[train_idx]
    id_test = ID_array[test_idx]

    train_data = X_train.copy()
    print(f'shape of train_data: {train_data.shape}')
    train_data['deltaG'] = y_train

    test_data = X_test.copy()
    test_data['deltaG'] = y_test

    fold_model_path = os.path.join(model_root_path, f'Fold{fold}')
    os.makedirs(fold_model_path, exist_ok=True)

    predictor = TabularPredictor(
        label='deltaG',
        path=fold_model_path,
        problem_type='regression',
        eval_metric='r2'
    ).fit(
        train_data=train_data,
        num_cpus=24,
        num_gpus=0,
        num_stack_levels=0,
        auto_stack=False,
        presets='good_quality',
        num_bag_folds=5,
    )

    y_train_pred = predictor.predict(train_data.drop(columns=['deltaG']))
    y_test_pred = predictor.predict(test_data.drop(columns=['deltaG']))

    train_rmse = mean_squared_error(y_train, y_train_pred) ** 0.5
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    train_r, _ = pearsonr(y_train, y_train_pred)

    test_rmse = mean_squared_error(y_test, y_test_pred) ** 0.5
    test_mae = mean_absolute_error(y_test, y_test_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    test_r, _ = pearsonr(y_test, y_test_pred)

    kf_results[fold] = {
        'Model': f'Fold_{fold}',
        'Train_RMSE': float(train_rmse),
        'Train_MAE': float(train_mae),
        'Train_R2': float(train_r2),
        'Train_PearsonR': float(train_r),
        'Test_RMSE': float(test_rmse),
        'Test_MAE': float(test_mae),
        'Test_R2': float(test_r2),
        'Test_PearsonR': float(test_r),
    }

    print(f"Fold {fold}: Train_R2 = {train_r2:.3f}, Test_R2 = {test_r2:.3f}, "
          f"Train_RMSE = {train_rmse:.3f}, Test_RMSE = {test_rmse:.3f}")

    train_result = pd.DataFrame({
        'ID': id_train,
        'True_deltaG': y_train,
        'Pred_deltaG': y_train_pred
    })
    test_result = pd.DataFrame({
        'ID': id_test,
        'True_deltaG': y_test,
        'Pred_deltaG': y_test_pred
    })

    mode = 'w' if (not os.path.exists(excel_out) and fold == 1) else 'a'
    with pd.ExcelWriter(excel_out, engine='openpyxl', mode=mode) as writer:
        train_result.to_excel(writer, sheet_name=f'train_fold{fold}', index=False)
        test_result.to_excel(writer, sheet_name=f'test_fold{fold}', index=False)

json_out = os.path.join(model_root_path, 'TrainingResults_5fold.json')
with open(json_out, 'w') as f:
    json.dump(kf_results, f, indent=2)

In [ ]:
# SHAP Analysis in each fold
import os
import json

import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from autogluon.tabular import TabularPredictor

model_root_path = (
    './AutoGluonModels/101Features_good_5bag_5fold'
)

n_splits_outer = 5

skf = StratifiedKFold(
    n_splits=n_splits_outer,
    shuffle=True,
    random_state=1000
)

print(f'Shape of selected_feat: {selected_X_feat.shape}')

type_labels = df['Type'].astype(str).values

max_display = 10

# ========= 2. 5-fold SHAP Analysis =========
for fold, (train_idx, test_idx) in enumerate(skf.split(selected_X_feat, type_labels), start=1):
    fold_id = fold
    print(f"\n===== SHAP for Fold {fold_id} =====")

    fold_model_path = os.path.join(model_root_path, f'Fold{fold_id}')
    predictor_bs = TabularPredictor.load(fold_model_path)


    shap_dir = os.path.join(fold_model_path, "SHAP")
    os.makedirs(shap_dir, exist_ok=True)

    X_train = selected_X_feat.iloc[train_idx].copy()
    X_test = selected_X_feat.iloc[test_idx].copy()
    y_train = y_array[train_idx]
    y_test = y_array[test_idx]
    id_train = ID_array[train_idx]
    id_test = ID_array[test_idx]

    train_data = X_train.copy()
    train_data['deltaG'] = y_train
    test_data = X_test.copy()
    test_data['deltaG'] = y_test

    leaderboard = predictor_bs.leaderboard(test_data, silent=True)

    X_bg = X_train.sample(n=min(200, len(X_train)), random_state=fold_id)
    X_explain = X_test
    feature_names = list(X_train.columns)

    def model_predict(x):
        if isinstance(x, pd.DataFrame):
            df_in = x.copy()
        else:
            df_in = pd.DataFrame(x, columns=feature_names)
        return predictor_bs.predict(df_in).to_numpy()


    explainer = shap.Explainer(
        model_predict,
        X_bg,
        seed=1000,
        feature_names=feature_names
    )


    n_features = X_bg.shape[1]
    shap_values = explainer(X_explain, max_evals=2 * n_features + 1)
    shap_array = shap_values.values  # (n_samples, n_features)
    feat_names = shap_values.feature_names

    np.save(
        os.path.join(shap_dir, f"shap_values_fold{fold_id}.npy"),
        shap_array
    )
    X_explain.to_parquet(
        os.path.join(shap_dir, f"X_explain_fold{fold_id}.parquet")
    )

    mean_abs = np.mean(np.abs(shap_array), axis=0)
    shap_importance = (
        pd.DataFrame({"feature": feat_names,
                      "mean_abs_shap": mean_abs})
        .sort_values("mean_abs_shap", ascending=False)
    )

    shap_importance.to_excel(
        os.path.join(shap_dir, f"SHAP_importance_fold{fold_id}.xlsx"),
        index=False
    )


    pd.DataFrame({"ID": id_test}).to_excel(
        os.path.join(shap_dir, f"Test_IDs_fold{fold_id}.xlsx"),
        index=False
    )

    plt.figure()
    shap.summary_plot(
        shap_array,
        X_explain,
        feature_names=feat_names,
        max_display=max_display,
        show=False
    )
    plt.tight_layout()
    plt.savefig(
        os.path.join(shap_dir, f"SHAP_summary_fold{fold_id}.png"),
        dpi=300,
        bbox_inches='tight'
    )
    plt.close()

    topN = max_display
    shap_bar_df = shap_importance.head(topN).iloc[::-1]

    plt.figure(figsize=(6, 5))
    plt.barh(
        shap_bar_df['feature'],
        shap_bar_df['mean_abs_shap']
    )
    plt.xlabel("Mean(|SHAP value|)")
    plt.title(f"SHAP Feature Importance (Fold {fold_id})")
    plt.tight_layout()
    plt.savefig(
        os.path.join(shap_dir, f"SHAP_barplot_fold{fold_id}.png"),
        dpi=300,
        bbox_inches="tight"
    )
    plt.close()

    print(f"Fold {fold_id}: SHAP arrays & importance saved to {shap_dir}")

In [ ]:
# SHAP Analysis for out-of fold
import os
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

model_root_path = (
    './AutoGluonModels/101Features_good_5bag_5fold'
)

fold_ids = [1, 2, 3, 4, 5]

all_shap = []
all_X = []

for fid in fold_ids:
    shap_dir = os.path.join(model_root_path, f'Fold{fid}', 'SHAP')

    shap_path = os.path.join(shap_dir, f"shap_values_fold{fid}.npy")
    X_path = os.path.join(shap_dir, f"X_explain_fold{fid}.parquet")

    shap_array = np.load(shap_path)  # (n_val_fold, n_features)
    X_explain_fold = pd.read_parquet(X_path)

    all_shap.append(shap_array)
    all_X.append(X_explain_fold)


shap_values_cv = np.vstack(all_shap)  # (N_total, n_features)
X_cv = pd.concat(all_X, axis=0, ignore_index=True)
assert shap_values_cv.shape[0] == X_cv.shape[0]
assert shap_values_cv.shape[1] == X_cv.shape[1]

feature_names = list(X_cv.columns)

max_display = 10

print("shap_values_cv shape:", shap_values_cv.shape)
print("X_cv shape:", X_cv.shape)

#SHAP summary（beeswarm）
plt.figure()
shap.summary_plot(
    shap_values_cv,
    X_cv,
    feature_names=feature_names,
    max_display=max_display,
    show=False
)
plt.tight_layout()
out_png = os.path.join(model_root_path, f"SHAP_summary_5fold_beeswarm.png")
plt.savefig(out_png, dpi=300, bbox_inches='tight', transparent=True)
plt.close()

print("Saved 5-fold SHAP summary beeswarm to:", out_png)

#SAHP bar summary
plt.figure()
shap.summary_plot(
    shap_values_cv,
    X_cv,
    feature_names=feature_names,
    max_display=max_display,
    plot_type="bar",
    show=False
)
plt.tight_layout()
out_png_bar = os.path.join(model_root_path, f"SHAP_summary_5fold_bar.png")
plt.savefig(out_png_bar, dpi=300, bbox_inches='tight', transparent=True)
plt.close()

print("Saved 5-fold SHAP summary bar to:", out_png_bar)


In [51]:
# Global SHAP importance+beewarm

def convert_rgb_color(x, y, z):
    return (x/255, y/255, z/255)

max_display = 10

mean_abs_shap = np.abs(shap_values_cv).mean(axis=0)  # shape: (n_features,)
sorted_idx = np.argsort(mean_abs_shap)[::-1]
top_idx = sorted_idx[:max_display]

top_features = [feature_names[i] for i in top_idx]
top_importance = mean_abs_shap[top_idx]

shap_top = shap_values_cv[:, top_idx]
X_top = X_cv[top_features]

feature_type_map = {
    # host-level
    "HostNonpolarSA": "host",
    "HostMPI": "host",
    "HostPositiveSA": "host",
    "HostPolarSA": "host",
    # guest-level
    "GuestLogP": "guest",
    "GuestMACCSKey_49": "guest",
    "GuestMACCSKey_128": "guest",
    "GuestNegativeSA": "guest",
    # complex-level
    "InteractionFP_5": "complex",
    "GuestCavityRatio": "complex",
}

type_color_map = {
    "host":    convert_rgb_color(230,111,81),
    "guest":   convert_rgb_color(233,196,107),
    "complex": convert_rgb_color(138,176,125),
    "other":   "#7f7f7f",
}

bar_colors = []
for f in top_features:
    ftype = feature_type_map.get(f, "other")
    bar_colors.append(type_color_map[ftype])

plt.rcParams.update({
    "font.size": 9,
    "axes.labelsize": 9,
    "axes.titlesize": 9,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "legend.fontsize": 8,
})

fig, axes = plt.subplots(
    1, 2,
    figsize=(18, 5),
    gridspec_kw={"width_ratios": [3.2, 3]},
    dpi=300
)

ax_bar = axes[0]
top_importance_plot = top_importance[::-1]
top_features_plot = top_features[::-1]
bar_colors_plot = bar_colors[::-1]
y_pos_plot = np.arange(len(top_features_plot))

face_colors = [(*c, 0.7) for c in bar_colors_plot]

bars = ax_bar.barh(
    y_pos_plot,
    top_importance_plot,
    color=face_colors,
    edgecolor="black",
    linewidth=0.8,
)

ax_bar.set_yticks(y_pos_plot)
ax_bar.set_yticklabels(top_features_plot)
ax_bar.set_xlabel("mean(|SHAP value|)")
ax_bar.set_title("Feature importance")

from matplotlib.patches import Patch

ax_bee = axes[1]
plt.sca(ax_bee)

shap.summary_plot(
    shap_top,
    X_top,
    feature_names=top_features,
    max_display=max_display,
    sort=False,
    show=False,
    # alpha=0.6,
    cmap="coolwarm"
)
ax_bee.set_title("Feature impact on ΔG")
ax_bar.spines["top"].set_visible(False)
ax_bar.spines["right"].set_visible(False)
ax_bar.spines["left"].set_visible(True)
ax_bar.spines["bottom"].set_visible(True)

ax_bee.set_yticklabels([])
ax_bee.set_ylabel("")
ax_bee.tick_params(axis="y", length=0)

plt.tight_layout()

out_joint = os.path.join(
    model_root_path,
    f"SHAP_5fold_importance_beeswarm.png"
)
fig.savefig(out_joint, dpi=300, bbox_inches="tight", transparent=False)
plt.close(fig)

print("Saved joint importance + beeswarm figure to:", out_joint)


Saved joint importance + beeswarm figure to: /mnt/4TBStorage1/Desktop/Papers/HostGuestModel/HostGuestModel_DataFile/BindingPrediction/ver_SupraBank/MLDeltaG/Training/Cleared3/AutoGluonModels_FineTune_MACCSKey/101Features_good_5bag_5fold_noSolvent/SHAP_5fold_importance_beeswarm_top10.png


In [53]:
# Local SHAP Analysis for Each Host Type
import os
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt

model_root_path = (
    './AutoGluonModels/101Features_good_5bag_5fold'
)

fold_ids = [1, 2, 3, 4, 5]

descriptor_excel = (
    './host_guest_features_SI.xlsx'
)

df_all = pd.read_excel(descriptor_excel, sheet_name='Features').reset_index(drop=True)
id2type = df_all.set_index('ID')['Type'].astype(str).to_dict()

all_shap = []
all_X = []
all_type = []

for fid in fold_ids:
    shap_dir = os.path.join(model_root_path, f'Fold{fid}', 'SHAP')

    shap_path = os.path.join(shap_dir, f"shap_values_fold{fid}.npy")
    X_path = os.path.join(shap_dir, f"X_explain_fold{fid}.parquet")
    shap_array = np.load(shap_path)  # (n_val_fold, n_features)
    X_explain_fold = pd.read_parquet(X_path)  # (n_val_fold, n_features)

    ids_path = os.path.join(shap_dir, f"Test_IDs_fold{fid}.xlsx")
    ids_fold = pd.read_excel(ids_path)['ID'].tolist()
    assert len(ids_fold) == shap_array.shape[0] == X_explain_fold.shape[0]

    types_fold = [id2type[i] for i in ids_fold]
    all_shap.append(shap_array)
    all_X.append(X_explain_fold)
    all_type.extend(types_fold)

shap_values_cv = np.vstack(all_shap)  # (N_total, n_features)
X_cv = pd.concat(all_X, axis=0, ignore_index=True)
type_cv = np.array(all_type)  # (N_total,)
assert shap_values_cv.shape[0] == X_cv.shape[0] == len(type_cv)
assert shap_values_cv.shape[1] == X_cv.shape[1]

feature_names = list(X_cv.columns)

print("Total samples:", X_cv.shape[0])
print("Num features (with PCA):", X_cv.shape[1])

out_root = os.path.join(model_root_path, "SHAP_ByHostType")
os.makedirs(out_root, exist_ok=True)

host_types = sorted(np.unique(type_cv))
print("Host types:", host_types)

topN = 5

for htype in host_types:
    mask = (type_cv == htype)
    shap_ht = shap_values_cv[mask, :]  # (n_ht_samples, n_features_non_pca)
    X_ht = X_cv[mask].reset_index(drop=True)

    if shap_ht.shape[0] == 0:
        continue

    print(f"\n===== HostType: {htype} =====")
    print("Samples:", shap_ht.shape[0])

    safe_htype = "".join(c if c.isalnum() else "_" for c in htype)

    mean_abs_ht = np.mean(np.abs(shap_ht), axis=0)
    shap_imp_ht = (
        pd.DataFrame({
            "feature": feature_names,
            "mean_abs_shap": mean_abs_ht
        })
        .sort_values("mean_abs_shap", ascending=False)
    )

    excel_out = os.path.join(
        out_root,
        f"SHAP_importance_5fold_{safe_htype}.xlsx"
    )
    shap_imp_ht.to_excel(excel_out, index=False)
    print("Saved importance table to:", excel_out)

    shap_bar_df = shap_imp_ht.head(topN).iloc[::-1]

    plt.figure(figsize=(6, 5))
    plt.barh(
        shap_bar_df['feature'],
        shap_bar_df['mean_abs_shap']
    )
    plt.xlabel("Mean(|SHAP value|)")
    plt.title(f"SHAP Feature Importance (HostType = {htype}, top {topN})")
    plt.tight_layout()
    png_bar = os.path.join(
        out_root,
        f"SHAP_bar_{safe_htype}_top{topN}.png"
    )
    plt.savefig(png_bar, dpi=300, bbox_inches="tight")
    plt.close()
    print("Saved bar plot to:", png_bar)

    plt.figure()
    shap.summary_plot(
        shap_ht,
        X_ht,
        feature_names=feature_names,
        max_display=topN,
        show=False,
        cmap='coolwarm'
    )
    plt.tight_layout()
    png_bee = os.path.join(
        out_root,
        f"SHAP_beeswarm_{safe_htype}_top{topN}.png"
    )
    plt.savefig(png_bee, dpi=300, bbox_inches='tight')
    plt.close()
    print("Saved beeswarm plot to:", png_bee)

Total samples: 876
Num features (with PCA): 101
Host types: [np.str_('Cucurbit[n]uril'), np.str_('Cyclodextrin'), np.str_('MetalCage'), np.str_('OrganicCage'), np.str_('Pillar[n]arene')]

===== HostType: Cucurbit[n]uril =====
Samples: 250
Saved importance table to: /mnt/4TBStorage1/Desktop/Papers/HostGuestModel/HostGuestModel_DataFile/BindingPrediction/ver_SupraBank/MLDeltaG/Training/Cleared3/AutoGluonModels_FineTune_MACCSKey/101Features_good_5bag_5fold_noSolvent/SHAP_ByHostType_noPCA/SHAP_importance_5fold_Cucurbit_n_uril.xlsx
Saved bar plot to: /mnt/4TBStorage1/Desktop/Papers/HostGuestModel/HostGuestModel_DataFile/BindingPrediction/ver_SupraBank/MLDeltaG/Training/Cleared3/AutoGluonModels_FineTune_MACCSKey/101Features_good_5bag_5fold_noSolvent/SHAP_ByHostType_noPCA/SHAP_bar_Cucurbit_n_uril_top5.png
Saved beeswarm plot to: /mnt/4TBStorage1/Desktop/Papers/HostGuestModel/HostGuestModel_DataFile/BindingPrediction/ver_SupraBank/MLDeltaG/Training/Cleared3/AutoGluonModels_FineTune_MACCSKey/10